In [ ]:
import torch
import matplotlib.pyplot as plt
# ^^^ pyforest auto-imports - don't write above this line
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm.auto import tqdm
from copy import deepcopy

from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP

from dataset import LUDB
from cfg import TrainCfg, ModelCfg
from trainer import LUDBTrainer
from model import ECG_UNET_LUDB
from metrics import compute_metrics

In [ ]:
TrainCfg.db_dir = "/home/wenhao/Jupyter/wenhao/data/LUDB/"

In [ ]:
train_cfg_fl = deepcopy(TrainCfg)
train_cfg_fl.use_single_lead = False
train_cfg_fl.loss = "FocalLoss"

train_cfg_ce = deepcopy(TrainCfg)
train_cfg_ce.use_single_lead = False
train_cfg_ce.loss = "CrossEntropyLoss"

In [ ]:
ds_train_fl = LUDB(train_cfg_fl, training=True, lazy=False)
ds_train_ce = LUDB(train_cfg_ce, training=True, lazy=False)

In [ ]:
# ds_train_fl._load_all_data()
# ds_train_ce._load_all_data()

In [ ]:
ds_val_fl = LUDB(train_cfg_fl, training=False, lazy=False)
ds_val_ce = LUDB(train_cfg_ce, training=False, lazy=False)

In [ ]:
# ds_val_fl._load_all_data()
# ds_val_ce._load_all_data()

## dry run: no augmentation, no preprocessing

In [ ]:
# train_config = deepcopy(TrainCfg)
model_config = deepcopy(ModelCfg)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = ECG_UNET_LUDB(model_config.n_leads, model_config)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = LUDBTrainer(
    model=model,
    model_config=model_config,
    train_config=train_cfg_fl,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train_fl, ds_val_fl)

In [ ]:
bmd = trainer.train()